# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     /home/randomwalk10/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/randomwalk10/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///message_categories.db')
df = pd.read_sql_table("message_categories_database", engine)

# split data into X(feature) and Y(class),
# with classes consisting of single label dropped as well
feature_names = ["id", "message", "original", "genre"]
X = df["message"].values
df = df.drop(feature_names, axis=1)
cls_to_drop = []
for label in df.columns:
    if len(np.unique(df[label]))<2:
        cls_to_drop.append(label)
df = df.drop(cls_to_drop, axis=1)
Y = df.values

# get list of column names for later use
col_names = df.columns.tolist()

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())    
    ])),

    ('clf', MultiOutputClassifier(MultinomialNB()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [6]:
# train pipeline
pipeline.fit(X_train, y_train);

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [8]:
# report f1 score, precision, and recall for each output category
list_accuracy_scores = []
for i, label in enumerate(col_names):
    target_names = [label+"_"+str(x) for x in np.unique(Y[:, i])]
    list_accuracy_scores.append(
        accuracy_score(y_test[:, i], y_pred[:, i]))
    print(classification_report(y_test[:, i], y_pred[:, i], target_names=target_names))

             precision    recall  f1-score   support

  related_0       0.80      0.04      0.07      1563
  related_1       0.76      1.00      0.86      4944
  related_2       0.00      0.00      0.00        47

avg / total       0.77      0.76      0.67      6554

             precision    recall  f1-score   support

  request_0       0.85      1.00      0.92      5443
  request_1       0.92      0.14      0.24      1111

avg / total       0.86      0.85      0.80      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6521
    offer_1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.76      0.85      0.80      3884
aid_related_1       0.74      0.60      0.66      2670

  avg / total       0.75      0.75      0.74      6554

                precision    recall  f1-score   support

medical_help_0       0

/home/randomwalk10/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
print("average accuracy score is", sum(list_accuracy_scores)/len(list_accuracy_scores))

average accuracy score is 0.9332970051004837


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None, 'steps': [('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])),
  ('clf',
   MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
              n_jobs=1))], 'text_pipeline': Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...y=None)), ('tfidf', Tf

In [11]:
# parameter grid
parameters = {
    'text_pipeline__vect__stop_words': (None, 'english'),
    'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__alpha': [0.5, 1, 2],
    'clf__estimator__fit_prior': [False, True]
}

# use grid search
cv = GridSearchCV(pipeline, parameters)
# train data
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'text_pipeline__vect__stop_words': (None, 'english'), 'text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__alpha': [0.5, 1, 2], 'clf__estimator__fit_prior': [False, True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# predict on test data
y_pred_cv = cv.predict(X_test)

In [13]:
# report f1 score, precision, and recall for each output category
list_accuracy_scores = []
for i, label in enumerate(col_names):
    target_names = [label+"_"+str(x) for x in np.unique(Y[:, i])]
    list_accuracy_scores.append(
        accuracy_score(y_test[:, i], y_pred_cv[:, i]))
    print(classification_report(y_test[:, i], y_pred_cv[:, i], target_names=target_names))

             precision    recall  f1-score   support

  related_0       0.67      0.52      0.59      1563
  related_1       0.86      0.92      0.89      4944
  related_2       0.68      0.49      0.57        47

avg / total       0.81      0.82      0.81      6554

             precision    recall  f1-score   support

  request_0       0.92      0.91      0.92      5443
  request_1       0.60      0.64      0.62      1111

avg / total       0.87      0.87      0.87      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6521
    offer_1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.81      0.71      0.76      3884
aid_related_1       0.64      0.76      0.70      2670

  avg / total       0.74      0.73      0.73      6554

                precision    recall  f1-score   support

medical_help_0       0

In [14]:
# print out average accuracy score
print("average accuracy score is", sum(list_accuracy_scores)/len(list_accuracy_scores))

average accuracy score is 0.9300623392475698


In [15]:
cv.best_params_

{'clf__estimator__alpha': 0.5,
 'clf__estimator__fit_prior': False,
 'text_pipeline__tfidf__use_idf': True,
 'text_pipeline__vect__stop_words': 'english'}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
# pipeline
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())    
    ])),

    ('clf', MultiOutputClassifier(SGDClassifier(
        loss='log', n_jobs=-1, random_state=42)))# use SVM
])

# grid parameters
parameters = {
    'text_pipeline__vect__stop_words': (None, 'english'),
    'text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__alpha': [0.0001, 0.001, 0.005],
    'clf__estimator__tol': [1e-3, 1e-4, 1e-5]
}

# use grid search
cv = GridSearchCV(pipeline, parameters)

# train model
cv.fit(X_train, y_train);

# predict
y_pred = cv.predict(X_test)

# check result
list_accuracy_scores = []
for i, label in enumerate(col_names):
    target_names = [label+"_"+str(x) for x in np.unique(Y[:, i])]
    list_accuracy_scores.append(
        accuracy_score(y_test[:, i], y_pred[:, i]))
    print(classification_report(y_test[:, i], y_pred[:, i], target_names=target_names))

             precision    recall  f1-score   support

  related_0       0.73      0.35      0.48      1563
  related_1       0.82      0.96      0.88      4944
  related_2       0.00      0.00      0.00        47

avg / total       0.79      0.81      0.78      6554

             precision    recall  f1-score   support

  request_0       0.90      0.98      0.94      5443
  request_1       0.84      0.47      0.61      1111

avg / total       0.89      0.90      0.88      6554

             precision    recall  f1-score   support

    offer_0       0.99      1.00      1.00      6521
    offer_1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

               precision    recall  f1-score   support

aid_related_0       0.78      0.86      0.82      3884
aid_related_1       0.76      0.64      0.70      2670

  avg / total       0.77      0.77      0.77      6554

                precision    recall  f1-score   support

medical_help_0       0

/home/randomwalk10/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
# print out average accuracy score
print("average accuracy score is", sum(list_accuracy_scores)/len(list_accuracy_scores))

average accuracy score is 0.9452591656131478


In [18]:
cv.best_params_

{'clf__estimator__alpha': 0.0001,
 'clf__estimator__tol': 0.001,
 'text_pipeline__tfidf__use_idf': False,
 'text_pipeline__vect__stop_words': 'english'}

### 9. Export your model as a pickle file

In [19]:
# save pipeline model
joblib.dump(cv.best_estimator_, "pipeline.pkl", compress=1)

['pipeline.pkl']

In [20]:
# load model from file
loaded_model = joblib.load("pipeline.pkl")
# measure the model
y_pred = loaded_model.predict(X_test)
list_accuracy_scores = []
for i, _ in enumerate(col_names):
    list_accuracy_scores.append(
        accuracy_score(y_test[:, i], y_pred[:, i]))
# print out average accuracy score
print("average accuracy score is", sum(list_accuracy_scores)/len(list_accuracy_scores))

average accuracy score is 0.9452591656131478


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.